In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_03 import *

In [3]:
path= Path('../PCImages')
path2fm= '../FeatureMatsMerged/TheGreatCollection.txt'
path2colnames= 'FeatureMatIndex.txt'

In [4]:
fm= fm_from_txt(path2fm, path2colnames)
fm.shape

(56657, 105)

In [5]:
cols_filtered=get_filtered_cols(fm)
len(cols_filtered)

44

## Mixup dataset

In [16]:
#export
from PIL import Image

In [6]:
#export
class NumpyToFloatTensor(object): 
    def __call__(self, img): return torch.from_numpy(img)

In [156]:
#export
class CellMixUpDataSet(Dataset):
    def __init__(self, path, fm, cols2keep, label = 'label1', mixup_cs=[.4, .3, .3], trfms=None, random_sample=None):
        self.fm = fm
        self.cols2keep= cols2keep
        self.y = self.fm[label]
        self.id2label= id2label(self.fm.id, self.y)
        self.all_data_files = get_filtered_files(path, img_ids=self.fm.id, labels=self.y) 
        self.data_files= self.all_data_files
        if random_sample is not None: self.data_files= random.sample(self.data_files, random_sample)

        self.trfms = get_trfms(trfms)
        self.cs= mixup_cs

    def __len__(self):
        return len(self.data_files)
    
    def get_row(self, img_id):
        x_np= np.array(self.fm.loc[self.fm['id'].isin([img_id])][self.cols2keep]).flatten() 
        return torch.cuda.FloatTensor(x_np)
    
    def __getitem__(self, i):
        img_name  =        self.data_files[i]
        img_id, phase =    split_fn(img_name)
        label=             self.id2label[img_id]
        img_names=         [f for f in self.all_data_files if img_id in split_fn(f)[0]]
        img1, img2, img3 = [io.imread(name) for name in img_names]
        image=             self.cs[0]*img1 + self.cs[1]*img2 + self.cs[2]*img3
        fm_row=            self.get_row(img_id)
        image=             Image.fromarray(np.uint8(image))
        if self.trfms:
            image = self.trfms(image)
        return image, fm_row, label

In [150]:
ds= CellMixUpDataSet(path, fm, cols_filtered, label = 'label1', 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(50,10, p=0.5))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(1.2704, device='cuda:0') tensor(0.6300, device='cuda:0')


In [155]:
ds= CellMixUpDataSet(path, fm, cols_filtered, label = 'label1', 
                mixup_cs=[.5, .25, .25],
                trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
                random_sample=1000)
learn= Learner(ds, CombinedModel(50,10, p=0.5))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(1.2347, device='cuda:0') tensor(0.6450, device='cuda:0')


In [157]:
!python notebook2script.py 04_mixup_dataset.ipynb

Converted 04_mixup_dataset.ipynb to exp/nb_04.py
